The dict type is not only widely used in our programs but also a fundamental part of the Python implementation. Module namespaces, class and instance attributes, and function keyword arguments are some of the fundamental constructs where dictionaries are deployed. The built-in functions live in \____builtins__\__.\____dict__\__.

Because of their crucial role, Python dicts are highly optimized. Hash tables are the engines behind Python’s high-performance dicts.

We also cover sets in this chapter because they are implemented with hash tables as well. Knowing how a hash table works is key to making the most of dictionaries and sets.
Here is a brief outline of this chapter:
* Common dictionary methods
* Special handling for missing keys
* Variations of dict in the standard library
* The set and frozenset types
* How hash tables work
* Implications of hash tables (key type limitations, unpredictable ordering, etc.)

# Generic Mapping Types

The collections.abc module provides the Mapping and MutableMapping ABCs to formalize the interfaces of dict and similar types (in Python 2.6 to 3.2, these classes are imported from the collections module, and not from collections.abc). See Figure 3-1 (See book).

Implementations of specialized mappings often extend dict or collections.UserDict, instead of these ABCs. The main value of the ABCs is documenting and formalizing the minimal interfaces for mappings, and serving as criteria for isinstance tests in code that needs to support mappings in a broad sense: 

In [ ]:
my_dict = {}
from collections import abc
isinstance(my_dict, abc.Mapping)

Using isinstance is better than checking whether a function argument is of dict type, because then alternative mapping types can be used.
All mapping types in the standard library use the basic dict in their implementation, so they share the limitation that the keys must be hashable (the values need not be hashable, only the keys).

## What is hashable?

Here is part of the definition of hashable from the Python Glossary:

*An object is hashable if it has a hash value which never changes during its lifetime (it needs a \____hash__\__() method), and can be compared to other objects (it needs an \____eq__\__() method). Hashable objects which compare equal must have the same hash value. […]*

The atomic immutable types (str, bytes, numeric types) are all hashable. A frozenset is always hashable, because its elements must be hashable by definition. A tuple is hashable only if all its items are hashable. See tuples tt, tl, and tf: 

In [ ]:
tt = (1, 2, (30, 40))
hash(tt)

In [ ]:
tl = (1, 2, [30, 40])
hash(tl)

In [ ]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

At the time of this writing, the Python Glossary states: “All of Python’s immutable built-in objects are hashable” but that is inaccurate because a tuple is immutable, yet it may contain references to unhashable objects.

User-defined types are hashable by default because their hash value is their id() and they all compare not equal. If an object implements a custom \____eq__\__ that takes into account its internal state, it may be hashable only if all its attributes are immutable.

Given these ground rules, you can build dictionaries in several ways. The Built-in Types page in the Library Reference has this example to show the various means of building a dict:

In [ ]:
a = dict(one = 1, two = 2, three = 3)
b = {'one' : 1, 'two' : 2, 'three' : 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three' : 3, 'one' : 1, 'two' : 2})
a == b == c == d == e

In addition to the literal syntax and the flexible dict constructor, we can use dict comprehensions to build dictionaries. See the next section.

# dict Comprehensions

Since Python 2.7, the syntax of listcomps and genexps was applied to dict comprehensions (and set comprehensions as well, which we’ll soon visit). A dictcomp builds a dict instance by producing key:value pair from any iterable. Example 3-1 shows the use of dict comprehensions to build two dictionaries from the same list of tuples.

In [ ]:
# Example 3-1. Examples of dict comprehensions
# A list of pairs can be used directly with the dict constructor.
DIAL_CODES = [
        (86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
    ]

# Here the pairs are reversed: country is the key, and code is the value.
country_code = {country: code for code, country in DIAL_CODES}
country_code

In [ ]:
# Reversing the pairs again, values uppercased and items filtered by code < 66.
{code: country.upper() for country, code in country_code.items() if code < 66}

If you’re used to liscomps, dictcomps are a natural next step. If you aren’t, the spread of the listcomp syntax means it’s now more profitable than ever to become fluent in it.
We now move to a panoramic view of the API for mappings.

# Overview of Common Mapping Methods

The basic API for mappings is quite rich. Table 3-1 (see book) shows the methods implemented by dict and two of its most useful variations: defaultdict and OrderedDict, both defined in the collections module. 

The way update handles its first argument m is a prime example of duck typing: it first checks whether m has a keys method and, if it does, assumes it is a mapping. Otherwise, update falls back to iterating over m, assuming its items are (key, value) pairs. The constructor for most Python mappings uses the logic of update internally, which means they can be initialized from other mappings or from any iterable object producing (key, value) pairs.

A subtle mapping method is setdefault. We don’t always need it, but when we do, it provides a significant speedup by avoiding redundant key lookups. If you are not comfortable using it, the following section explains how, through a practical example.

## Handling Missing Keys with setdefault

In line with the fail-fast philosophy, dict access with d[k] raises an error when k is not an existing key. Every Pythonista knows that d.get(k, default) is an alternative to d[k] whenever a default value is more convenient than handling KeyError. However, when updating the value found (if it is mutable), using either \____getitem__\__ or get is awkward and inefficient. Consider Example 3-2, a suboptimal script written just to show one case where dict.get is not the best way to handle a missing key.

Example 3-2 is adapted from an example by Alex Martelli,2 which generates an index like that in Example 3-3.

In [ ]:
# Example 3-2. index.py uses dict.get to fetch and update 
# a list of word occurences from the index

In [ ]:
import sys
import re

WORD_RE = re.compile('\w+')

index = {}
with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            # Get the list of occurrences for word, or [] if not found.
            occurrences = index.get(word, [])
            # Append new location to occurrences
            occurrences.append(location)
            # Put changed occurrences into index dict; this entails a second search through the index.
            index[word] = occurrences       

# print in alphabetical order
# In the key= argument of sorted I am not calling str.upper, just passing a reference to that method so the sorted function can use it to normalize the words for sorting.
for word in sorted(index, key=str.upper):  
    print(word, index[word])

The three lines dealing with occurrences in Example 3-2 can be replaced by a single line using dict.setdefault. Example 3-4 is closer to Alex Martelli’s original example.

In [ ]:
# Example 3-4. index.py uses dict.setdefault to fetch and update a list of word occurrences from the index in a single line; contrast with Example 3-2.

import sys
import re

WORD_RE = re.compile('\w+')

index = {}
with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # Get the list of occurrences for word, or set it to [] if not found; setdefault returns the value, so it can be updated without requiring a second search.
            index.setdefault(word, []).append(location)  

# print in alphabetical order
for word in sorted(index, key=str.upper):
    print(word, index[word])

In other words, the end result of this line…

*my_dict.setdefault(key, []).append(new_value)*

…is the same as running…

*if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)*

…except that the latter code performs at least two searches for key—three if it’s not found—while setdefault does it all with a single lookup.

A related issue, handling missing keys on any lookup (and not only when inserting), is the subject of the next section. 

# Mappings with Flexible Key Lookup

Sometimes it is convenient to have mappings that return some made-up value when a missing key is searched. There are two main approaches to this: one is to use a defaultdict instead of a plain dict. The other is to subclass dict or any other mapping type and add a \____missing__\__ method. Both solutions are covered next.

## defaultdict: Another Take on Missing Keys

Example 3-5 uses collections.defaultdict to provide another elegant solution to the problem in Example 3-4. A defaultdict is configured to create items on demand whenever a missing key is searched.

Here is how it works: when instantiating a defaultdict, you provide a callable that is used to produce a default value whenever \____getitem__\__ is passed a nonexistent key argument.

For example, given an empty defaultdict created as dd = defaultdict(list), if 'new-key' is not in dd, the expression dd['new-key'] does the following steps:
1. Calls list() to create a new list.
2. Inserts the list into dd using 'new-key' as key.
3. Returns a reference to that list.

The callable that produces the default values is held in an instance attribute called default_factory.

In [ ]:
# Example 3-5. index_default.py: using an instance of defaultdict instead of the setdefault method
import sys
import re
import collections

WORD_RE = re.compile('\w+')

# Create a defaultdict with the list constructor as default_factory.
index = collections.defaultdict(list)     
with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # If word is not initially in the index, 
            # the default_factory is called to produce 
            # the missing value, which in this case is an 
            # empty list that is then assigned to index[word] and returned, so the .append(location) operation always succeeds.
            index[word].append(location)  

# print in alphabetical order
for word in sorted(index, key=str.upper):
    print(word, index[word])

If no default_factory is provided, the usual KeyError is raised for missing keys.

The mechanism that makes defaultdict work by calling default_factory is actually the __missing__ special method, a feature supported by all standard mapping types that we discuss next. 

## The __missing__ Method

Underlying the way mappings deal with missing keys is the aptly named \____missing__\__ method. This method is not defined in the base dict class, but dict is aware of it: if you subclass dict and provide a \____missing__\__ method, the standard dict.\____getitem__\__ will call it whenever a key is not found, instead of raising KeyError.

The \____missing__\__ method is just called by \____getitem__\__ (i.e., for the d[k] operator). The presence of a \____missing__\__ method has no effect on the behavior of other methods that look up keys, such as get or \____contains__\__ (which implements the in operator). This is why the default_factory of defaultdict works only with \____getitem__\__, as noted in the warning at the end of the previous section.

Suppose you’d like a mapping where keys are converted to str when looked up. A concrete use case is the Pingo.io project, where a programmable board with GPIO pins (e.g., the Raspberry Pi or the Arduino) is represented by a board object with a board.pins attribute, which is a mapping of physical pin locations to pin objects, and the physical location may be just a number or a string like "A0" or "P9_12". For consistency, it is desirable that all keys in board.pins are strings, but it is also convenient that looking up my_arduino.pin[13] works as well, so beginners are not tripped when they want to blink the LED on pin 13 of their Arduinos. Example 3-6 shows how such a mapping would work.

In [ ]:
# Example 3-6. When searching for a nonstring key, StrKeyDict0 converts it to str when it is not found

import collections

# StrKeyDict0 inherits from dict
class StrKeyDict(collections.UserDict):  # <1>

    def __missing__(self, key):  # <2>
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data  # <3>

    def __setitem__(self, key, item):
        self.data[str(key)] = item   # <4>